Extraccion de palabras clave de los resumenes las paginas web de la carpeta train:

Se realiza la extraccion de las palabras clave de los resumenes de todas las paginas web por clase, primero usando Beautiful Soup para la extraccion del contenido, Sumy para el resumen y luego Keybert para la extraccion de las palabras clave.

In [5]:
import os
from bs4 import BeautifulSoup
import nltk
#nltk.download('punkt')
from nltk.corpus import stopwords
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from keybert import KeyBERT
import json

El siguiente codigo realiza lo siguiente:

1. Se define una funcion para extraer un resumen de cada pagina web, usando en primer lugar Beautiful Soup para la extraccion del texto.
2. Luego se realiza la sumarizacion utilizando Sumy y se genera un resumen de cada pagina con una longitud de 5 oraciones.
3. Y por ultimo se procesan todas las paginas web de cada clase y se guarda el resumen extraido en una lista.

In [58]:

# Ruta de la carpeta que contiene los archivos origen
folder_path = "C:/Users/usuario/Downloads/dataset_splitted/splits/train/faculty"  

# Lista creada para almacenar el texto del cuerpo de todas las páginas
all_page_summary = []

# Función para extraer el resumen del cuerpo de la página
def extract_summary(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:
        text = file.read()

    # Se analiza el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')

    # Se extrae el texto del cuerpo de la página
    page_text = ' '.join(soup.stripped_strings)
    page_text = page_text.replace('\n', '')

     # Se hace la sumarización utilizando sumy
    parser = PlaintextParser.from_string(page_text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, sentences_count=5)  # Se puede ajustar la cantidad de oraciones en el resumen.

    summary_text = ' '.join([str(sentence) for sentence in summary])
    
    return summary_text

# Se procesan los archivos y se guarda el texto del cuerpo en la lista all_page_texts
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):  
        summary_text = extract_summary(file_path)
        all_page_summary.append(summary_text)


En esta parte se usa Keybert para poder generar las palabras clave de todos los textos extraidos de las paginas web y que se almacenaron en la lista.

In [59]:
#Se usa keybert para obtener las palabras clave de todos los textos.
modelo = KeyBERT()
palabras_clave = modelo.extract_keywords(all_page_summary, stop_words='english')

#Ver las palabras clave encntradas
for keyword in palabras_clave:
    print(keyword)
    
todas_palabras_clave = [clave for lista_p_claves in palabras_clave for clave in lista_p_claves]

[('reconstructing', 0.4572), ('recognition', 0.4219), ('vision', 0.3866), ('motion', 0.3754), ('algorithms', 0.3449)]
[('performance', 0.2588), ('embedded', 0.2585), ('visualization', 0.2445), ('workload', 0.24), ('simulation', 0.2281)]
[('programming', 0.4898), ('mathematics', 0.4115), ('syllabus', 0.3279), ('distributedreal', 0.3264), ('graphics', 0.2726)]
[('eecs', 0.4199), ('edu', 0.4087), ('university', 0.391), ('electrical', 0.3603), ('engineering', 0.355)]
[('programming', 0.4982), ('program', 0.4121), ('software', 0.3704), ('gmtcontent', 0.3669), ('graphics', 0.3568)]
[('computational', 0.4689), ('multimedia', 0.4584), ('digital', 0.4571), ('programming', 0.4024), ('lightness', 0.3557)]
[('linguistics', 0.4757), ('logic', 0.4171), ('computational', 0.4105), ('computing', 0.3927), ('programming', 0.3083)]
[('web', 0.3543), ('cs', 0.3541), ('technology', 0.3065), ('information', 0.2752), ('1996', 0.2728)]
[('telecommunications', 0.4654), ('rica', 0.3759), ('1996', 0.3561), ('1997

Para poder tener la palabra una sola vez, se calcula la suma del valor de cada palabra cuando esta varias veces, para esto se usa un diccionario.

In [60]:
# Se usa un diccionario para poder calcular la suma del valor de cada palabra clave cuando se encuentra varias veces.
sums = {}

# Se itera sobre las palabras clave encontradas y se suman sus valores
for clave in todas_palabras_clave:
    palabra, valor = clave  
    if palabra in sums:
        sums[palabra] += valor
    else:
        sums[palabra] = valor

# Se ordenan las palabras clave encontradas por el resultado de sus sumas de manera descendente
suma_ordenada = sorted(sums.items(), key=lambda x: x[1], reverse=True)

# Imprimir las palabras clave con sus sumas
for palabra, suma in suma_ordenada:
    print(f"Palabra clave: {palabra}, Suma: {suma}")


Palabra clave: computing, Suma: 61.60060000000001
Palabra clave: programming, Suma: 49.1848
Palabra clave: cs, Suma: 47.3721
Palabra clave: algorithms, Suma: 38.691
Palabra clave: computational, Suma: 27.927599999999998
Palabra clave: parallel, Suma: 24.5113
Palabra clave: edu, Suma: 19.050399999999996
Palabra clave: computer, Suma: 17.6319
Palabra clave: software, Suma: 16.9399
Palabra clave: distributed, Suma: 16.4847
Palabra clave: compilers, Suma: 13.8838
Palabra clave: apache, Suma: 12.895100000000003
Palabra clave: faculty, Suma: 11.6525
Palabra clave: gmtserver, Suma: 11.4957
Palabra clave: ai, Suma: 11.459999999999997
Palabra clave: graphics, Suma: 10.677300000000004
Palabra clave: complexity, Suma: 10.608299999999996
Palabra clave: courses, Suma: 10.2738
Palabra clave: systems, Suma: 10.271599999999998
Palabra clave: teaching, Suma: 10.054400000000001
Palabra clave: database, Suma: 9.9415
Palabra clave: compiler, Suma: 9.852
Palabra clave: ieee, Suma: 9.3716
Palabra clave: rob

Por ultimo se genera una lista ordenada con todas las palabras clave del resumen de todas las paginas de cada clase y se guardan en un archivo JSON.

In [61]:
# Se crea una lista para todas las palabras clave
palabras_clave_lista = [palabra for palabra, suma in suma_ordenada]

# Ruta de la carpeta de salida para el archivo JSON
output_folder = "C:/Users/usuario/Downloads/dataset_splitted/splits/train_json/faculty_json"
os.makedirs(output_folder, exist_ok=True)

# Ruta completa al archivo JSON en la carpeta de salida
output_json_file = os.path.join(output_folder, "keywords2.json")

# Se crea un diccionario para almacenar las palabras y el ground_truth
data_json = {"words": palabras_clave_lista, "ground_truth": "faculty"}

# Se guarda el diccionario en un archivo JSON 
with open(output_json_file, 'w', encoding='utf-8') as json_file:
    json.dump(data_json, json_file, ensure_ascii=False, separators=(',', ':'))

print(f"La lista de palabras clave se ha guardado en '{output_json_file}'")



La lista de palabras clave se ha guardado en 'C:/Users/usuario/Downloads/dataset_splitted/splits/train_json/faculty_json\keywords2.json'


In [37]:
all_page_summary[9]

"Date: Tue, 26 Nov 1996 03:56:00 GMTServer: Apache/1.1.1Content-type: text/htmlContent-length: 637Last-modified: Sun, 27 Oct 1996 00:02:37 GMT Yakov Keselman's Home Page Yakov Keselman's Home Page I am a Graduate Student (3-d year) in the Dept of Computer Science at Rutgers University . e-mail: keselman@paul.rutgers.edu Phone: (908) 445-0050 Various applets Lots of other info about myself"